In [60]:
import pandas as pd
import os
import requests
import base64
import spotipy
from lyricsgenius import Genius
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

In [58]:
load_dotenv()

SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GENIUS_ACCESS_TOKEN = os.getenv("GENIUS_ACCESS_TOKEN")

In [16]:
# Load dataset to pandas Dataframe
df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv", low_memory=False)

In [3]:
# Drop rows that do not contain spotify song id and chords
mask = df["chords"].notna() & df["spotify_song_id"].notna() & df["genres"].notna()

clean_df = df[mask].copy()

In [5]:
print(clean_df.loc[1, "chords"])

<intro_1> E D A/Cs E D A/Cs <verse_1> E D A/Cs E D A/Cs E D A/Cs E D A C <chorus_1> E G D A E G D A E G D A C D E D A/Cs <verse_2> E D A/Cs E D A/Cs E D A/Cs E D A C <chorus_2> E G D A E G D A E G D A C D <bridge_1> E C G D E C G D E C G D C D E G E G D A E G D A E G D A C D <chorus_3> E G D A E G D A E G D A C D <bridge_2> E C G D E C G D E C G D C D E


In [6]:
type(clean_df.loc[1, "chords"])

str

In [79]:
# Get Spotify API token (it constantly refreshes after every 3600s)
auth_manager = SpotifyClientCredentials(
    client_id=SPOTIFY_CLIENT_ID, 
    client_secret=SPOTIFY_CLIENT_SECRET
)

# initialize spotipy client
sp = spotipy.Spotify(auth_manager=auth_manager)

# initialize genius client
genius = Genius(GENIUS_ACCESS_TOKEN)

In [80]:
# Test search for summer by keshi
mask = df["spotify_song_id"] == "0LqCvXVNvcnkq78sCQsoO0"
request_full_track_data = sp.track(clean_df.loc[mask, 'spotify_song_id'].values[0])

# test requesting full track data from spotify api
# request_full_track_data = sp.track(clean_df.loc[1, 'spotify_song_id'])

In [82]:
def get_lyrics(spotify_song_id):
    request_full_track_data = sp.track(spotify_song_id)
    song = genius.search_song(request_full_track_data["name"], request_full_track_data["artists"][0]["name"])
    return song.lyrics

In [83]:
print(get_lyrics("0LqCvXVNvcnkq78sCQsoO0"))

Searching for "summer" by keshi...
Done.
[Intro]
Feel like summer and I don't wanna miss you
If we don’t touch lips, not an issue
I don’t wanna talk 'bout nobody else
Know I said no love but I want to
Feel like summer and I don't wanna miss you
If we don’t touch lips, not an issue
I don’t wanna talk 'bout nobody else
Know I said no love but I want to

[Verse 1]
One time, maybe twice in
One night, it’s alright we’re
Both alone so go together
Kiss my wounds but not forever
Fuck around, feel my heartbeat
Fallin' down in the backseat
Fill me up 'cause I'm runnin' on empty
And it’s fine if we’re only pretending

[Pre-Chorus]
Three months is all we got
Try not to fall in love
Don’t think I've had enough but I said it’s fine by me

[Chorus]
Feel like summer and I don't wanna miss you
If we don’t touch lips, not an issue
I don’t wanna talk 'bout nobody else
Know I said no love but I want to
Feel like summer and I don't wanna miss you
If we don’t touch lips, not an issue
I don’t wanna talk 'bou